In [28]:
import pandas as pd
import json
from os import listdir
from os.path import isfile, join

In [29]:
englishMasculineWordsList = open('./data/gendered_words_lists/en/masculine.json', 'r')
englishMasculineWords = json.load(englishMasculineWordsList)
englishFeminineWordsList = open('./data/gendered_words_lists/en/feminine.json', 'r')
englishFeminineWords = json.load(englishFeminineWordsList)

In [31]:
# Count the number of gendered words in each job advertisement
dataDirectory = './data/job_advertisements/en/'
# get the list of file names
files = [f for f in listdir(dataDirectory) if isfile(join(dataDirectory, f)) and ('failed' not in f)]

# declare the dataframe object to save the results
df = pd.DataFrame(index=['index'], columns=['dataIndex', 'occupation', 'masclineWordCount', 'feminineWordCount', 'wordCount'])
dfByOccupation = pd.DataFrame(index=['index'], columns=['occupation', 'masclineWordCount', 'feminineWordCount', 'wordCount'])
genderedWordDf = pd.DataFrame(index=['index'], columns=['word', 'occupation'])

dataIndex = 0
for file in files:
    # open job advertisements file
    jsonOpen = open(dataDirectory + file, 'r')
    jobAdvertisements = json.load(jsonOpen)
    occupation = file.replace('.json', '')
    
    masculineWordCounterPerOccupation = 0
    feminineWordCounterPerOccupation = 0
    wordCounterPerOccupation = 0

    dfIndex = 1
    for jobAdvertisement in jobAdvertisements:
        if jobAdvertisement.get(str(dfIndex)):
            if jobAdvertisement[str(dfIndex)]['title'] is None or jobAdvertisement[str(dfIndex)]['description'] is None :
                break
            title = jobAdvertisement[str(dfIndex)]['title'].replace('\n','').replace('(',' ').replace(',',' ').replace('-',' ').replace(')',' ').strip().lower()
            description = jobAdvertisement[str(dfIndex)]['description'].replace('\n','').replace(',',' ').replace('-',' ').replace('(',' ').replace(')',' ').strip().lower()
            all_sentences = title + ' ' + description

            masculineWordCounterPerAdvertisement = 0
            feminineWordCounterPerAdvertisement = 0
            counter = 0 
            for word in all_sentences.split(' '):
                counter += 1
                for englishMasculineWord in englishMasculineWords:
                    if englishMasculineWord.lower() == word or ( '*' in englishMasculineWord and englishMasculineWord.lower().replace('*', '') in word):
                        dataIndex += 1
                        masculineWordCounterPerAdvertisement += 1
                        wordData = pd.DataFrame({
                            'word': englishMasculineWord,
                            'occupation': occupation,
                        }, index=[str(counter)])
                        genderedWordDf = pd.concat([genderedWordDf, wordData])
                        break
                for englishFeminineWord in englishFeminineWords:
                    if englishFeminineWord.lower() == word or ( '*' in englishFeminineWord and englishFeminineWord.lower().replace('*', '') in word):
                        dataIndex += 1
                        feminineWordCounterPerAdvertisement += 1
                        wordData = pd.DataFrame({
                            'word': englishFeminineWord,
                            'occupation': occupation,
                        }, index=[str(counter)])
                        genderedWordDf = pd.concat([genderedWordDf, wordData])
                        break

            record =  pd.DataFrame({
                'dataIndex': dataIndex,
                'occupation': occupation,
                'masclineWordCount': masculineWordCounterPerAdvertisement,
                'feminineWordCount': feminineWordCounterPerAdvertisement,
                'wordCount': counter
            }, index=[str(dfIndex)])
            df = pd.concat([df, record])
            masculineWordCounterPerOccupation += masculineWordCounterPerAdvertisement
            feminineWordCounterPerOccupation += feminineWordCounterPerAdvertisement
            wordCounterPerOccupation += counter
        dfIndex += 1

    record =  pd.DataFrame({
        'occupation': occupation,
        'masclineWordCount' : masculineWordCounterPerOccupation,
        'feminineWordCount' : feminineWordCounterPerOccupation, 
        'wordCount' : wordCounterPerOccupation
    }, index=[str(dataIndex)])
    dfByOccupation = pd.concat([dfByOccupation, record])

df.to_csv('./result/english_result.csv')
dfByOccupation.to_csv('./result/english_result_per_occupation.csv')
genderedWordDf.to_csv('./result/english_gendered_words_in_advertisements.csv')
